## Imports/Setup

In [9]:
%run ../00_functions/00_custom_functions.ipynb
standard_setup(extra_packages=True, verbose=False)
data_ingestion_prep(start_yearmonth='1979-01', end_yearmonth='2022-12', verbose=False)

env: ESMFMKFILE=/home/ds4114/.conda/envs/leap_test202308-3/lib/esmf.mk
Files will be outputed as: .nc
ECMWF CDS API key already installed


In [10]:
from xgboost import XGBRegressor

In [ ]:
#optionally turn off some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Set Parameters

In [41]:
#Set location of input files (path from root above)
#Note that these were processed such that they already share a coordinate set
sst_processed  = global_vars['download_folder'] + 'SST/processed/SST_NOAA_OI-V2-1x1_198201-202304.nc'
sss_processed  = global_vars['download_folder'] + 'SSS/processed/SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_198201-202304.nc' #previously 202303
mld_processed  = global_vars['download_folder'] + 'MLD/processed/MLD_IFREMER-deBoyer_DT02-c1m-1x1_198201-202304.nc'
chl_processed  = global_vars['download_folder'] + 'CHL/processed/CHL_ARI-ST-GlobColour_L3m-GLOB-100-merged-GSM-CHL1_198201-202304.nc'
pco2_processed = global_vars['download_folder'] + 'pCO2/processed/pCO2_LEAP_SOCAT-ERA5-fco2-weighted-gridded_198201-202212.nc' #updated for fco2
xco2_processed = global_vars['download_folder'] + 'xCO2/processed/xCO2_NOAA_xCO2-mm-gl-monthly_198201-202304.nc'
#add additional sources if desired
sst_processed_option2 = global_vars['download_folder'] + 'SST/processed/SST_ECMWF_ERA5-monthly-reanalysis-1x1-SST_198201-202304.nc'
sst_processed_option3 = global_vars['download_folder'] + 'SST/processed/SST_JMA_JRA55-do-monthly-reanalysis-SST_198201-202304.nc'
mld_processed_option2 = global_vars['download_folder'] + 'MLD/processed/MLD_UCSD-Argo_MLD-dt-mean-1x1_198201-202304.nc'

gobms = global_vars['download_folder'] + 'GOBM/processed/GOBM_GCB-2023_fco2-8-models_195901-202212.nc'
netmask = global_vars['download_folder'] + 'Masks/processed/mask_LEAP_land-sea-network.nc'
gobm_list = ['CNRM-ESM2-1', 'FESOM2-REcoM', 'IPSL', 'MOM6-Princeton', 'MRI-ESM2-2','NorESM-OC1.2', 'CESM-ETHZ', 'ACCESS']

list_for_df = [sst_processed, sss_processed, mld_processed, chl_processed, pco2_processed, xco2_processed
              ,sst_processed_option2, sst_processed_option3, mld_processed_option2
              ,gobms, netmask
              ] 

In [27]:
#This is where we set parameters for the ML algorithm for finding the long term pco2 mean feature

#The next variable is for the XGBoost method for both pCO2 Residual and creating the long term pCo2 mean feature. They were determined via a grid search in previous iterations. 
best_params = {'max_depth': 9, 'n_estimators': 1000} 
random_seed = 47  #Set the random seeds used for training
jobs = -1         #Number of cores you have access to for model training; -1 for all available ones

#This variable is a list of features used for the Long Term pCO2 mean machine learning
feature_sel = ['sst','sst_anomaly','sss','sss_anomaly','chl_log','chl_log_anomaly','mld_log','xco2_trend','A','B','C','T0','T1']
target_sel = ['fco2']  #previously was pco2
sst_variable_option = ['sst']  #the name of the SST variable to use for calculating the residual component of the target

## Full Set of Base Features

In [29]:
xrfull = xr.merge([xr_open_dataset_custom(f) for f in list_for_df], compat='broadcast_equals')
xrfull = xrfull.sel(time=slice(str(processed_start_yearmonth),str(processed_end_yearmonth)))
xrfull.attrs = "" #just removing attribute details since wont be accurate anymore
#xrfull

In [31]:
#next add derived (logs)
xrfull = xrfull.assign( mld_log = log_or_0_xr(xrfull.mld, 'mld_log') 
                       ,chl_log = log_or_0_xr(xrfull.chl, 'chl_log')
                       ,mld_argo_log = log_or_0_xr(xrfull.mld_argo, 'mld_argo_log')
                      )

In [32]:
%%time
#add anomalies fields
anomalies = xrfull.groupby("time.month") - xrfull.groupby("time.month").mean("time")
anomalies = anomalies.get(['sst','sss','chl_log', 'sst_era5', 'sst_jra55']) #just need SST, SSS, CHL anomalies
anomalies = anomalies.drop('month') 
anomalies = anomalies.rename({ 'sst': 'sst_anomaly'
                              ,'sss':'sss_anomaly'
                              ,'chl_log':'chl_log_anomaly'
                              ,'sst_era5':'sst_era5_anomaly'
                              ,'sst_jra55':'sst_jra55_anomaly'
                             }) 
#anomalies
xrfull = xrfull.merge(anomalies, compat='identical')

CPU times: user 5.58 s, sys: 2.09 s, total: 7.68 s
Wall time: 7.69 s


In [33]:
#add time and space derivations
xrfull = xrfull.assign( days_idx = xrfull.time.dt.dayofyear 
                       ,lon_rad = np.radians(xrfull.xlon)
                       ,lat_rad = np.radians(xrfull.ylat)
                      )
xrfull = xrfull.assign( T0 = np.cos(xrfull.days_idx * 2 * np.pi / 365)
                       ,T1 = np.sin(xrfull.days_idx * 2 * np.pi / 365)
                       ,A  = np.sin(xrfull.lat_rad)
                       ,B  = np.cos(xrfull.lat_rad)*np.sin(xrfull.lon_rad)
                       ,C  = -np.cos(xrfull.lat_rad)*np.cos(xrfull.lon_rad)
                      )

## Learning Long Term pCO2 Mean

#### Set Data Fields/Points For Learning Long Term Mean

In [34]:
%%time
xr_for_ocean_co2_mean = xrfull.get(feature_sel + target_sel)  #Note - we train on all available data (no time slice is used here)
df_for_ocean_co2_mean = xr_for_ocean_co2_mean.to_dataframe() #expensive function; need lots of RAM 
df_for_ocean_co2_mean_to_train = df_for_ocean_co2_mean[(~df_for_ocean_co2_mean.isna().any(axis=1))]  #only keep points that are not null 
print(f'Number of points in time/space for training: {df_for_ocean_co2_mean_to_train.shape[0]}')
#df_for_ocean_co2_mean_to_train

df_for_ocean_co2_mean_to_predict_temp = df_for_ocean_co2_mean.loc[:,feature_sel]
df_for_ocean_co2_mean_to_predict = df_for_ocean_co2_mean_to_predict_temp[(~df_for_ocean_co2_mean_to_predict_temp.isna().any(axis=1))]  #only predict on points with all variables globally
print(f'Number of points in time/space available to reconstruct: {df_for_ocean_co2_mean_to_predict.shape[0]}')
#df_for_ocean_co2_mean_to_predict

Number of points in time/space for training: 312201
Number of points in time/space available to reconstruct: 18198122
CPU times: user 2.93 s, sys: 2.39 s, total: 5.31 s
Wall time: 5.32 s


#### ML for Ocean CO2_Long_Term_Mean Feature

In [35]:
%%time
X_train = df_for_ocean_co2_mean_to_train.loc[:,feature_sel]
y_train = df_for_ocean_co2_mean_to_train.loc[:,target_sel]

model = XGBRegressor(random_state=random_seed, **best_params, n_jobs=jobs)
print(f'Training started on '+datetime.datetime.now().strftime('%Y-%m-%d %H:%M')+'...')
model.fit(X_train, y_train)    #training on all data with no cross validation because we are only calculating a long term average
                               #Model evalation for pco2 residual is in the next script
print('Training completed on '+datetime.datetime.now().strftime('%Y-%m-%d %H:%M')+'. Predicting...')
ocean_co2_for_mean_recon = model.predict(df_for_ocean_co2_mean_to_predict)
print("Complete")  #Training and predicting may take 15 minutes

Training started on 2023-12-17 23:45...


/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstanc

Training completed on 2023-12-17 23:45. Predicting...


/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/ds4114/.conda/envs/leap_test202310-4/lib/python3.11/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstanc

Complete
CPU times: user 56.2 s, sys: 450 ms, total: 56.6 s
Wall time: 1.97 s


In [36]:
#average across time and add back to dataset 
ocean_co2_for_mean_recon_xr = pd.DataFrame(ocean_co2_for_mean_recon,index=df_for_ocean_co2_mean_to_predict.index,columns=['ocean_co2_recon_for_mean']).to_xarray()
xrfull = xrfull.merge(ocean_co2_for_mean_recon_xr, compat='identical') #add back to full set
xrfull = xrfull.assign(ocean_co2_mean = xrfull.ocean_co2_recon_for_mean.mean('time'))

In [22]:
#Export if desired. Some extra code to find the dates used
if False:
    min_yearmonth = str(ocean_co2_for_mean_recon_xr.time.min().data.astype('datetime64[s]').item().strftime('%Y%m')) #just gets the min date from the xarray in YYYYMM format
    max_yearmonth = str(ocean_co2_for_mean_recon_xr.time.max().data.astype('datetime64[s]').item().strftime('%Y%m')) 
    ocean_co2_long_term_mean = ocean_co2_for_mean_recon_xr.mean('time')
    ocean_co2_long_term_mean = ocean_co2_long_term_mean.rename({'ocean_co2_recon_for_mean': str(target_sel[0])+'_mean'})
    ocean_co2_output_name = 'pCO2_LEAP_XGBoost-'+str(target_sel[0])+'-long-term-mean-from-'+min_yearmonth+'-to-'+max_yearmonth+'.nc'
    output_xarray_with_date(ocean_co2_long_term_mean, global_vars['reconstruction_folder'], ocean_co2_output_name, with_date=False,  filetype=output_file_type)

Saved pCO2_LEAP_XGBoost-fco2-long-term-mean-from-198201-to-202303.nc to /data/artemis/workspace/ds4114/reconstructions/


## Residual (pCO2 T and NonT Features)

In [37]:
xrfull = xrfull.assign(ocean_co2_T = xrfull['ocean_co2_mean'] * np.exp(0.0413 * (xrfull[sst_variable_option[0]] - xrfull[sst_variable_option[0]].mean("time"))) ) # Wanninkhof et al. 2022
    #previously: xrfull = xrfull.assign(ocean_co2_T = xrfull['ocean_co2_mean'] * np.exp(0.0423* (xrfull.sst - xrfull.sst.mean("time"))) ) # Takahashia et al, 2004
xrfull = xrfull.assign(ocean_co2_nonT = xrfull[target_sel[0]] - xrfull.ocean_co2_T)

In [38]:
#Perform some clean up before exporting

xrfull = xrfull.drop_vars(['ocean_co2_recon_for_mean'])  #optionally could keep/drop this variable
xrfull = xrfull.rename({'ocean_co2_T': str(target_sel[0])+'_T'
                        ,'ocean_co2_nonT': str(target_sel[0])+'_nonT'
                        ,'ocean_co2_mean': str(target_sel[0])+'_mean'
                        #,'ocean_co2_recon_for_mean': str(target_sel[0])+'_recon_for_mean'
                       })

#add attributes as needed here
xrfull.attrs['description'] = "fCO2-Residual dataset for machine learning"
xrfull.attrs['formula_for_co2_residual'] = 'uses 0.0413 factor from Wanninkhof et al. 2022 and temperature variable: '+ str(sst_variable_option[0])
xrfull.attrs['training_features_for_co2_mean'] = str(feature_sel)
xrfull.attrs['hyperparameters_for_co2_mean'] = str(best_params)
xrfull.attrs['created'] = str(datetime.datetime.now())
#xrfull.attrs['methodology'] = "Bennington et al. (2022), JAMES"

In [50]:
#filter variables, if desired
xr_for_residual = xrfull.get(feature_sel + target_sel + ['days_idx','lon_rad','lat_rad','pco2','mld','chl','fco2_mean','sst_era5','sst_jra55','sst_era5_anomaly','sst_jra55_anomaly','mld_argo','mld_argo_log','fco2_T','fco2_nonT','net_mask'])
xr_for_hpd = xrfull.get(feature_sel + target_sel + gobm_list + ['mld','chl','fco2_mean','net_mask', 'days_idx','lon_rad','lat_rad'])

In [24]:
#export out (may be 3GB unless some variables dropped)
#naming conventions may need updating
if True:
    output_xarray_with_date(xrfull, global_vars['reconstruction_folder']+'', 'pCO2-HPD_LEAP_'+str(target_sel[0])+'-full-dataset-preML', filetype=output_file_type)
    output_xarray_with_date(xr_for_residual, global_vars['reconstruction_folder']+'', 'pCO2_LEAP_'+str(target_sel[0])+'-residual-dataset-preML', filetype=output_file_type)
    output_xarray_with_date(xr_for_hpd, global_vars['reconstruction_folder']+'', 'HPD_LEAP_fco2-GOBM-dataset-preML', filetype=output_file_type)

Saved pCO2_LEAP_fco2-residual-full-dataset-preML_198201-202304.nc to /data/artemis/workspace/ds4114/reconstructions/
